<a href="https://colab.research.google.com/github/gracefacetseng/mrc-image-converter/blob/main/MRC_and_Image_File_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image File Converter

When converting mrc files, it may be useful to have napari open to visualize the original file.

### 1. Install and import libraries:

In [42]:
import sys
!{sys.executable} -m pip install pathlib mrcfile Pillow numpy tifffile

from google.colab import files
import os
import mrcfile
from PIL import Image as im
import numpy as np
import tifffile as tif

### 2. Import your file and set conversion parameters:

Note: 'jpeg' and 'tif' will not work as inputs for the destination file type. Please be mindful of this, and any typos, as the code is written to exit the program if it takes an unknown input.

In [43]:
#import file
filename = files.upload()

for fn in filename.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(filename[fn])))

filetype = os.path.splitext(fn)[1]
print(fn)

# check input filetype
valid_filetypes = ['.png', '.tif', '.tiff', '.mrc', '.jpg', '.jpeg']

if filetype in valid_filetypes:
    pass
else:
    print('File type not accepted.')
    sys.exit()

#specify destination filetype
to_file = input('Convert to (png/tiff/jpg):')

def suffix(to_file):
    if to_file == 'png':
        return '.png'
    elif to_file == 'tiff':
        return '.tiff'
    elif to_file == 'jpg':
        return '.jpg'
    else:
        print('File type not accepted. Please try again.')
        sys.exit()

new_name = input('New file name:') + suffix(to_file)

Saving 22feb15b_ApoF_00005gr_00047sq_v02_00002hln_00014enn-a-DW.mrc to 22feb15b_ApoF_00005gr_00047sq_v02_00002hln_00014enn-a-DW (1).mrc
User uploaded file "22feb15b_ApoF_00005gr_00047sq_v02_00002hln_00014enn-a-DW.mrc" with length 67109888 bytes
22feb15b_ApoF_00005gr_00047sq_v02_00002hln_00014enn-a-DW.mrc
Convert to (png/tiff/jpg):png
New file name:conv


### 3. Convert your image to an array:

In [44]:
def img_to_array(fn):
    if filetype == '.mrc':
        return mrcfile.mmap(fn, permissive=True).data
    else:
        return np.asarray(im.open(fn))

array = img_to_array(fn)

### 4. Set contrast limits. Details in the code:

In [45]:
# to find the min and max of the array, if necessary
print(np.min(array))
print(np.max(array))

16037.959
21506.455


In [46]:
#set contrast limits
if filetype == '.mrc':
    contrast_min = 18432 # or a specified minimum pixel value
    contrast_max = 20017 # or a specified maximum pixel value

    ## contrast limits of mrc files must be manually adjusted to appear as they look in napari. in rare cases,
    ## the limits are equal to the min and max of the image. if so, use the same parameters for all other image
    ## file types. napari does not seem to adjust the contrast limits in any substantial way; these values may
    ## be found in metadata associated with the image itself. for now, the only way to find these values is by
    ## trial and error. it may be helpful to find the min and max values using the block above. contrast limits
    ## are often somewhere in the middle of that range.

else:
    contrast_min = array.min()
    contrast_max = array.max()

array = np.where(array <= contrast_min, contrast_min, array)
array = np.where(array >= contrast_max, contrast_max, array)
array = (array-np.min(array))/(np.max(array)-np.min(array))
array *= 255/array.max()

### 5. Convert the array to the desired image file and return output:

In [47]:
#convert array to new image filetype
def array_to_img(array):
    if to_file == 'tiff':
        array = array.astype(np.uint8)
        return tif.imwrite(new_name, array)
    elif to_file == 'png':
        array = array.astype(np.uint8)
        png = im.fromarray(array)
        return png.save(new_name, Format='PNG')
    else:
        array = array.astype(np.uint8)
        jpg = im.fromarray(array).convert('RGB')
        return jpg.save(new_name, Format='JPG')

array_to_img(array)
files.download(new_name)

print(fn, ' successfully converted to ', to_file,', see file ', new_name, sep='')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

22feb15b_ApoF_00005gr_00047sq_v02_00002hln_00014enn-a-DW.mrc successfully converted to png, see file conv.png
